In [2]:
from ultralytics import YOLO

Convert Annotation files from xml to txt

In [2]:
import xml.etree.ElementTree as ET
import os

def convert(size, box):
    dw = 1. / size[0]
    dh = 1. / size[1]
    x = (box[0] + box[1]) / 2.0 - 1
    y = (box[2] + box[3]) / 2.0 - 1
    w = box[1] - box[0]
    h = box[3] - box[2]
    x = x * dw
    w = w * dw
    y = y * dh
    h = h * dh
    return (x, y, w, h)

def convert_annotation(xml_file, output_dir):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    size = root.find('size')
    w = int(size.find('width').text)
    h = int(size.find('height').text)

    for obj in root.iter('object'):
        cls = obj.find('name').text
        if cls != "plate":
            continue
        xmlbox = obj.find('bndbox')
        b = (int(xmlbox.find('xmin').text), int(xmlbox.find('xmax').text), int(xmlbox.find('ymin').text), int(xmlbox.find('ymax').text))
        bb = convert((w, h), b)
        with open(f"{output_dir}/{os.path.basename(xml_file).replace('.xml', '.txt')}", 'w') as f:
            f.write(f"0 {bb[0]} {bb[1]} {bb[2]} {bb[3]}\n")

# Convert all XML annotations
xml_dir = 'annotations/test'
output_dir = 'labels/test'
os.makedirs(output_dir, exist_ok=True)

for xml_file in os.listdir(xml_dir):
    if xml_file.endswith('.xml'):
        convert_annotation(os.path.join(xml_dir, xml_file), output_dir)


In [19]:
# Check if GPU is available
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

Using device: cuda


Create YAML file to direct data path

In [ ]:
data_yaml = """
train: C:\Users\Alvino Angelo\Documents\UMD\DATA612\ALPR\images\train
val: C:\Users\Alvino Angelo\Documents\UMD\DATA612\ALPR\images\test
nc: 1
names: ['plate']
"""

with open('data.yaml', 'w') as f:
    f.write(data_yaml)


Model training

In [3]:
from ultralytics import YOLO

# Load a YOLOv8 model
model = YOLO('yolov8n.pt')

# Train the model
model.train(data='data.yaml', epochs=100, imgsz=640, batch=16, device=device, project='runs/train', name='alpr_model')


NameError: name 'device' is not defined

Load Model and test on new image

In [3]:
model = YOLO('runs\\train\\alpr_model\\weights\\best.pt')
results = model('IMG-20180108-WA0076.jpg')
results[0].show()


image 1/1 c:\Users\Alvino Angelo\Documents\UMD\DATA612\ALPR\IMG-20180108-WA0076.jpg: 384x640 1 plate, 65.5ms
Speed: 2.0ms preprocess, 65.5ms inference, 65.5ms postprocess per image at shape (1, 3, 384, 640)
